In [ ]:
import polars as pl

In [ ]:
weather_forecast = pl.scan_parquet("data/met_forecast.parquet")
weather_forecast.head(5).collect()

In [ ]:
weather_forecast.collect_schema()

In [ ]:
weather_nowcast = pl.scan_parquet("data/met_nowcast.parquet")
weather_nowcast.head(
    5
).collect()  # = weather_nowcast.tz_localize('UTC').tz_convert('CET') # Converting to local time

In [ ]:
windpower = pl.scan_parquet("data/wind_power_per_bidzone.parquet").rename(
    {"__index_level_0__": "time"}
)
windpower.head(5).collect()  #  = windpower.tz_localize('UTC').tz_convert('CET')

In [ ]:
windparks = pl.scan_csv("data/windparks_bidzone.csv", try_parse_dates=True).filter(
    pl.col("eic_code") == pl.col("eic_code").first().over("substation_name")
)
windparks.group_by("substation_name").agg(n=pl.count("eic_code")).sort("n").collect()

In [ ]:
windparks.filter(pl.col("substation_name") == "Måkaknuten").collect()

In [ ]:
windparks.group_by("bidding_area").agg(
    num_stations=pl.count("eic_code"),
    total_power_max=pl.col("operating_power_max").sum(),
    mean_power_max=pl.col("operating_power_max").mean(),
    min_valid_date=pl.col("prod_start_new").max(),
).collect()

In [ ]:
# Modelling this bid zone
bid_zone = "ELSPOT NO2"

# Selecting the windparks in bid zone from metadata
_windparks_in_bid_zone = windparks.filter(pl.col("bidding_area") == bid_zone)

# Selecting the windpower from bid zone
_windpower_in_bid_zone = windpower.select("time", bid_zone)

# Caclulcating the mean observed weather for the windparks in the bid zone
_weather_nowcast_in_bid_zone = (
    weather_nowcast.join(
        _windparks_in_bid_zone,
        left_on="windpark",
        right_on="substation_name",
        how="inner",
    )
    # .filter(pl.col("windpark").is_in(_windparks_in_bid_zone["substation_name"].implode()))
    .group_by("time")
    .agg(
        mean_wind_speed=pl.col("wind_speed_10m").mean(),
        median_wind_speed=pl.col("wind_speed_10m").median(),
        std_wind_speed=pl.col("wind_speed_10m").std(),
        wind_speed_weighted=(
            pl.col("wind_speed_10m") * pl.col("operating_power_max")
        ).sum()
        / pl.col("operating_power_max").sum(),
    )
)

# Concatenating datasets (weather and power) into one dataframe
data_bidzone = _windpower_in_bid_zone.join(
    _weather_nowcast_in_bid_zone, on="time"
)  # pd.concat([_windpower_in_bid_zone, _weather_nowcast_in_bid_zone], axis=1)

# Filtering out data where not all windparks are operational
min_valid_date = (
    _windparks_in_bid_zone.select(pl.col("prod_start_new").max()).collect().item()
)
data_bidzone = data_bidzone.filter(pl.col("time") > min_valid_date).drop_nulls()

In [ ]:
import plotly.express as px

px.scatter(
    data_bidzone.collect(),
    "median_wind_speed",
    bid_zone,
    color="std_wind_speed",
    opacity=0.5,
    height=700,
)

In [ ]:
from scipy.optimize import curve_fit
import numpy as np


def logistic_curve(x, L: float, k: float, x0: float):
    if isinstance(x, np.ndarray):
        return L / (1 + np.exp(-k * (x - x0)))
    else:
        return L / (1 + pl.exp(-k * (x - x0)))


xdata = data_bidzone.select("wind_speed_weighted").collect().to_numpy()[:, 0]
ydata = data_bidzone.select(bid_zone).collect().to_numpy()[:, 0]

res = curve_fit(logistic_curve, xdata, ydata, p0=[1000, 1, 6])
res

In [ ]:
hourly_station = (
    weather_nowcast.join(
        _windparks_in_bid_zone,
        left_on="windpark",
        right_on="substation_name",
        how="inner",
    )
    .filter(pl.col("time") > pl.col("prod_start_new"))
    .select("time", "windpark", "wind_speed_10m", "operating_power_max")
    .collect()
    .pivot("windpark", index="time", values="wind_speed_10m")
    .drop_nulls()
    .join(_windpower_in_bid_zone.collect(), on="time")
    .drop("time")
)

In [ ]:
px.histogram(
    weather_nowcast.join(
        _windparks_in_bid_zone,
        left_on="windpark",
        right_on="substation_name",
        how="inner",
    )
    .filter(pl.col("time") > pl.col("prod_start_new"))
    .select("time", "windpark", "wind_speed_10m", "operating_power_max")
    .with_columns(log_wind=(pl.col("wind_speed_10m")) ** (1 / 3))
    .collect(),
    "log_wind",
    facet_col="windpark",
    facet_col_wrap=4,
    height=1000,
)

In [ ]:
parks = hourly_station.drop(bid_zone).columns
park_weights = {
    x["substation_name"]: x["operating_power_max"]
    for x in _windparks_in_bid_zone.select("substation_name", "operating_power_max")
    .collect()
    .to_dicts()
}


def P(x, L: float, k: float, x0: float):
    total_p = 0
    for i, park in enumerate(parks):
        total_p += park_weights[park] * logistic_curve(x[:, i], L, k, x0)
    return total_p

In [ ]:
xdata = hourly_station.select(parks).to_numpy()
ydata = hourly_station.select(bid_zone).to_numpy()[:, 0]

res = curve_fit(P, xdata, ydata, p0=[1, 1, 6])
L, k, x0 = res[0]

In [ ]:
print(L, k, x0)

In [ ]:
hourly_station = hourly_station.with_columns(pred=P(xdata, L, k, x0))

In [ ]:
px.scatter(
    hourly_station.with_columns(mean_wind=pl.mean_horizontal(parks)).unpivot(
        [bid_zone, "pred"], index="mean_wind"
    ),
    "mean_wind",
    "value",
    color="variable",
)

## Nordpool API

In [ ]:
windpower.select(pl.col("time").min()).collect().item().isoformat()

In [ ]:
import requests

areas = [
    {"name": "NO1", "code": "10YNO-1--------2"},
    {"name": "NO2", "code": "10YNO-2--------T"},
    {"name": "NO3", "code": "10YNO-3--------J"},
    {"name": "NO4", "code": "10YNO-4--------9"},
    # {"name": "NO5", "code": "10Y1001A1001A48H"},
]
url = "https://ummapi.nordpoolgroup.com/messages"

messages = []
skip = 0
while True:
    res = requests.get(
        url,
        params={
            "limit": 2000,
            # "messageTypes": "ProductionUnavailability",
            "areas": [a["code"] for a in areas],
            # "fuelTypes": 19,
            # "publicationStartDate": "2020-01-01T00:00:00",
            "skip": skip,
        },
    )
    if res.status_code != 200:
        print(res.status_code)
        break

    content = res.json()
    if len(content["items"]) == 0:
        break
    messages.extend(content["items"])
    skip += len(content["items"])
    print(
        f"Retrieved: {len(content['items'])} ---- Progress: {skip}/{content['total']}"
    )
    if skip >= content["total"]:
        break

In [ ]:
import pandas as pd

messages = pd.json_normalize(messages).sort_values("publicationDate")

In [ ]:
messages["generationUnits"].dropna().iloc[1000]

In [ ]:
url = "https://ummapi.nordpoolgroup.com/infrastructure/fueltypes"
res = requests.get(url)
res.json()  # ["items"][0]

In [ ]:
windparks.filter(pl.col("eic_code") == "50WP00000001720X").collect()

## Modelling